# ** K-Nearest Neighbors Classifier **

A simple Data Science model, the KNN Classifier predicts the class of unknown data points by referencing them against its "neighbors," the data points closest to it. This model makes use of a breast cancer dataset and determines whether or not a tumor is malignant.

#### First, lets import all of our dependencies.

In [1]:
import numpy as np 
from math import sqrt
import pandas as pd
import random
from statistics import mode
import os
print("All dependencies successfully imported.")

All dependencies successfully imported.


#### Next, we need to change our working directory so our program can find the breast cancer dataset.

In [2]:
os.chdir('/Users/Sam/Documents/Python/Datasets')

#### Now, the data needs to be read into a dataframe, and organized into a suitable format. Most importantly, we need to ensure that the feature data and the label data is separated.

In [3]:
#Reading in the data set to a dataframe
orig_data = pd.read_csv('breast_cancer.txt')

#Next, separate the data into the features (predictors) and labels (classes: malignant or benign)
data = orig_data.drop(['class', 'patient_id'], axis = 1)
label = orig_data['class']

#Now we need to replace missing values in the data with the number zero.
data['bare_nuclei'].replace('?',0.0, inplace = True)
data['bare_nuclei'] = data['bare_nuclei'].astype('float')

#Convert both features (data) and labels
data = list(data.values)
label = list(label)

#Reference dictionary for easy to read results. 
dict = {2:'benign',4:'malignant'}

# With the data ready for analysis, let's begin to construct our model

## **Creating a comparison engine:** 
The point of this model is to find the neighbors of a given data point, those that are most similar. However, how can we determine which points are similar? The two most common functions are a Euclidian Distance Function and a Pearson Correlation Coefficient (r). 

This method is an **estimate** of the Pearson Coefficient
\begin{align}
r = \frac{{}\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \overline{x})^2(y_i - overline{y})^2}}
\end{align}

The **exact Pearson Coefficient** formula is shown below**
\begin{align}
\rho = \frac{\text{cov}(X,Y)}{\sigma_x \sigma_y}
\end{align}

**Where cov(X,Y) is equal to**
\begin{align}
cov(X,Y) = \sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})
\end{align}

**and**
\begin{align}
\sigma{_x} = \sqrt{\sum_{i=1}^{n}(x_i - \bar{x})^2\quad}
\end{align}

However, the **Euclidian** distance function is the simplest
to understand, and will be used in our model.
\begin{align}
e = \sqrt{\sum_{i=1}^n (x_i-y_i)^2}
\end{align}


Where x is a record (**person**) and y is another. Below is our euclidian function; this will calculate the distance between two different records and tell us how similar they are (smaller number = greater similarity).

In [4]:
#Must sort from smallest to largest to get similarity rankings.
def euclid(x,y):
    sums = 0.0
    for i in range(len(x)):
        sums += pow((x[i] - y[i]),2)

    return sqrt(sums)

At this point, we need to make functions to analyze the data. In order to do so effectively, we have to create a train and test set of data. The function below approximately splits the data into two samples with sizes specified by the user.

In [5]:
#Split data to train and test samples
#The random.random() function decides whether an item goes into the train or test set
#train_count and test_count keep track of how many records are in each sample
#x_traintest acts as a reference bank for all testing data later. It will be what "unknown" ...
#data points are compared against
def train_test_split(data,label, test_ratio):
    x_traintest = []
    x_test = []
    y_test = []
    train_count = 0
    test_count = 0
    for i in range(len(data)):
        if random.random() < test_ratio:
            test_count += 1
            x_test.append(data[i])
            y_test.append(label[i])
        else:
            train_count += 1
            x_traintest.append((data[i],label[i]))
            
    return x_traintest, x_test, y_test

#### Finally, the bottom two functions implement the euclidian distance engine we built to determine the closest neighbors for each point in the testing set. 

The getNeighbors() function below takes our reference bank of training data (x_traintest) and determines what points are closest neighbors to an unknown data point (testInstance) from our testing dataset. The similarity input determines what operator to use to determine similarity (in this case, we use euclid()). Lastly, k is the number of neighbors an item has. This number **MUST** be odd as this prevents a bimodal similarity sample (equal number of different neighbors).

In [6]:
# For each record in the training dataset, this determines a euclidian score
# These scores are sorted, and the labels  of the most similar k values
# Are returned (e.g. (5, Benign), (13.5, Malignant), etc.)

def getNeighbors(data,testInstance,similarity,k):
    neighbor = []
    for i in range(len(data)):
        neighbor.append(((similarity(data[i][0],testInstance)),data[i][1]))
    
    neighbor.sort(key=lambda tup: tup[0]) 
    
    neighbors = pd.DataFrame()
    neighbors['euclid'] = [x[0] for x in neighbor]
    neighbors['label'] = [x[1] for x in neighbor]
    
    return neighbors['label'][:k].tolist()

Since the function above has to be called for every testing data point, the get_results function is used to iterate through all data points in the testing set instead of just one. The inputs of this function are fed into the getNeighbors function, and a raw resultlist of predictions is given (e.g. benign, malignant, etc.)

In [7]:
def get_results(x_traintest,x_test,k):
    res_list = []
    label = [x[1] for x in x_data]
    for i in range(len(x_test)):
        kNeighbor = getNeighbors(x_traintest, x_test[i],euclid,k)
        res_list.append(mode(kNeighbor))
    return res_list

## With our analysis now finished, we need to write functions that will let us view the model's results and performance 

The view_results() function returns a dataframe with three columns: The actual result for a person's cancer diagnosis, the predicted result, and whether or not the prediction was correct (True or False). This makes it easy for us to see what the program actually ran.

In [8]:
def view_results(actual,predicted):
    result = pd.DataFrame()
    for i in range(len(actual)):
        actual[i] = dict[actual[i]]
        predicted[i] = dict[predicted[i]]
    result['ACTUAL'] = actual
    result['PREDICTED'] = predicted
    result['CORRECT?'] = [actual[i] == predicted[i] for i in range(len(actual))]
    
    return result

Moreover, we can see how accurate we are using the get_accuracy() function. This will tell us our accuracy as a percent, the raw number of cases we got correct, and the raw number of cases we got incorrect.

In [9]:
def get_accuracy(actual, predicted):
    count = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            count += 1
    num_correct = count
    num_incorrect = len(actual) - count
    return ("Accuracy: " + str(count / len(actual)*100) + " percent" + '\n' + 
           "Number correct: " + str(num_correct) + '\n' + 
           "Number incorrect: " + str(num_incorrect))

## By calling the functions below, we can view our results and see how good our model is.

In [10]:
#Makes testing data sample 20% of the entire dataset. This can be adjusted.
#For the purposes of this model, this is the training step.
x_data, x_test, y_test = train_test_split(data,label,0.2)

#This tests the model and returns a list of results
predicted_results = get_results(x_data,x_test,5)

#Organizing our results so we can see 20 of our test cases
viewed_results = view_results(y_test,predicted_results)
viewed_results = viewed_results[:20]

#This allows us to see the results
print(viewed_results)
print()
print(get_accuracy(y_test,predicted_results))

       ACTUAL  PREDICTED CORRECT?
0      benign     benign     True
1      benign  malignant    False
2      benign     benign     True
3      benign     benign     True
4      benign  malignant    False
5      benign     benign     True
6      benign     benign     True
7   malignant  malignant     True
8   malignant  malignant     True
9   malignant  malignant     True
10     benign     benign     True
11     benign     benign     True
12     benign     benign     True
13     benign     benign     True
14     benign     benign     True
15     benign     benign     True
16     benign     benign     True
17     benign     benign     True
18  malignant  malignant     True
19  malignant  malignant     True

Accuracy: 97.98657718120806 percent
Number correct: 146
Number incorrect: 3
